In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 50)
import math
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report

from IPython.display import display

import spacy
import re
#from nltk.corpus import gutenberg, stopwords
#from collections import Counter
#import nltk
#nltk.download('gutenberg')

#from spacy.lang.en import English

# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
warnings.filterwarnings(
    action="ignore",
    module="scipy",
    message="^internal gelsd"
)

from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.decomposition import PCA
from sklearn import neighbors
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

from sklearn import ensemble
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.preprocessing import LabelEncoder, Imputer
from sklearn.model_selection import train_test_split

from timeit import default_timer as timer

from nltk.tokenize import BlanklineTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
import urllib.request
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

import os

from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from sklearn.cluster import MiniBatchKMeans
from sklearn.datasets.samples_generator import make_blobs
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.cluster import SpectralClustering
from sklearn.cluster import AffinityPropagation
from sklearn import metrics
from itertools import cycle

import pydotplus
from sklearn import tree
from sklearn import preprocessing
%matplotlib inline
sns.set_style('white')

#### Multi-page scraper
Below is a spider that will go through a forum page with 100 threads, collect the urls for each thread, and then go to each thread's first page and collect the title, url, first post's text, original posting date, and how many posts were in the thread.  Note that, for some reason, the threads in the JSON document are NOT in the order they appear on the forum page...

In [2]:
# Importing in each cell because of the kernel restarts.
import scrapy
from scrapy.crawler import CrawlerProcess


class PFforumSpider(scrapy.Spider):
    # Naming the spider is important if you are running more than one spider of
    # this class simultaneously.
    name = "PFforum"
    
    # URL(s) to start with.
    start_urls = [
        'https://paizo.com/community/forums/pathfinder/pathfinderRPG/general',
    ]

    # set up a parse to go through and collect all the thread urls, then open them one by one and scrape their data
    def parse(self, response):
        for href in response.xpath("//table/tbody[//b]/tr/td/a[b]/@href").extract():
            url = response.urljoin(href)
            yield scrapy.Request(url, callback = self.parse_dir_contents)

    # set up a parse to scrape the threads' individual data 
    # example link: https://paizo.com/threads/rzs2k2y8?Best-Open-Content
    def parse_dir_contents(self, response):
        for sel in response.xpath('//td[@id="main-slot"]'):
            yield {
                'title': sel.xpath('//h1[a]/a/text()').extract(),
                'link': sel.xpath('//h1[a]/a/@href').extract(),
                'first_post': sel.xpath("normalize-space(string(//*[@itemprop='commentText']))").extract(),
                'first_post_date': sel.xpath('//span/a/time/text()').extract_first(),
                'thread_length': sel.xpath("normalize-space(string(//td[@class='tiny']))").extract()
                # Note that "thread_length" will be null if it's less than 51, as the marker the xpath expression
                # is reading doesn't appear unless there are more than 50 comments in the thread
            }

# Tell the script how to run the crawler by passing in settings.
process = CrawlerProcess({
    'FEED_FORMAT': 'json',         # Store data in JSON format.
    'FEED_URI': 'pf_forum_scrape.json',  # Name our storage file.
    'LOG_ENABLED': False,           # Turn off logging for now.
    'ROBOTSTXT_OBEY': True,
    'USER_AGENT': 'ThinkfulDataScienceBootcampCrawler (thinkful.com)',
    'AUTOTHROTTLE_ENABLED': True,
    'HTTPCACHE_ENABLED': True
})

# Start the crawler with our spider.
process.crawl(PFforumSpider)
process.start()
print('Success!')


Success!


In [3]:
import pandas as pd

firstpage = pd.read_json('pf_forum_scrape.json')
print(firstpage.shape)
print(firstpage.head())

(100, 5)
                                          first_post  \
0  [Hey guys it’s gonna be my first time being a ...   
1  [Really. it can do eveyrthing. with the human/...   
2  [In the Bible the original Horseman Of Famine ...   
3  [I think this has come up in almost every grou...   
4  [What are some rules appropriate ways to allow...   

               first_post_date  \
0  Wed, Apr 24, 2019, 01:13 pm   
1  Tue, Apr 23, 2019, 04:13 pm   
2  Thu, Apr 25, 2019, 01:19 pm   
3  Fri, Apr 12, 2019, 01:05 pm   
4   Mon, Apr 8, 2019, 02:16 pm   

                                                link    thread_length  \
0  [https://paizo.com/threads/rzs42jji?Making-a-m...               []   
1  [https://paizo.com/threads/rzs42jii?Is-it-me-o...  [1 to 50 of 56]   
2  [https://paizo.com/threads/rzs42jkm?Scales-for...               []   
3  [https://paizo.com/threads/rzs42j5s?How-do-you...               []   
4  [https://paizo.com/threads/rzs42j0z?Making-a-v...  [1 to 50 of 60]   

          

In [13]:
firstpage.loc[3,'link']

['https://paizo.com/threads/rzs42j5s?How-do-you-handle-high-explosives']

#### Single-page scraper

Below is a spider that will scrape the main forum page for titles, links, original post dates, and latest post dates for each thread.

In [2]:
# Importing in each cell because of the kernel restarts.
import scrapy
from scrapy.crawler import CrawlerProcess


class PFforumSpider(scrapy.Spider):
    # Naming the spider is important if you are running more than one spider of
    # this class simultaneously.
    name = "PFforum"
    
    # URL(s) to start with.
    start_urls = [
        'https://paizo.com/community/forums/pathfinder/pathfinderRPG/general',
    ]
            
    # Use XPath to parse the response we get.
    def parse(self, response):
        
        # Iterate over every <article> element on the page.
        for thread in response.xpath('//table/tbody[//b]'):

            # Yield a dictionary with the values we want.
            yield {
                # This is the code to choose what we want to extract
                # You can modify this with other Xpath expressions to extract other information from the site
                'thread_title': thread.xpath('tr/td/a/b/text()').extract_first(),
                'link': thread.xpath('tr/td/a[b]/@href').extract_first(),
                'original_post_date': thread.xpath('tr/td/div/span/time/text()').extract_first(),
                'newest_post_date': thread.xpath('tr/td/a/time/text()').extract_first()
                }

# Tell the script how to run the crawler by passing in settings.
process = CrawlerProcess({
    'FEED_FORMAT': 'json',         # Store data in JSON format.
    'FEED_URI': 'pf_forum_firstpage.json',  # Name our storage file.
    'LOG_ENABLED': False           # Turn off logging for now.
})

# Start the crawler with our spider.
process.crawl(PFforumSpider)
process.start()
print('Success!')


Success!


In [3]:
import pandas as pd

firstpage = pd.read_json('pf_forum_firstpage.json')
print(firstpage.shape)
print(firstpage.head())

(100, 4)
                                                link  \
0  https://paizo.com/threads/rzs2k2y8?Best-Open-C...   
1      https://paizo.com/threads/rzs2l3b0?FAQ-System   
2  https://paizo.com/threads/rzs42jqj?What-is-you...   
3     https://paizo.com/threads/rzs42jok?Blood-Hexes   
4  https://paizo.com/threads/rzs42jnt?SoVigilante...   

                                                name        newest_post_date  \
0                                  Best Open Content   Mar 6, 2018, 05:38 pm   
1                                         FAQ System  Feb 17, 2018, 08:18 am   
2         What is your top 10-20 low level monsters?          12 minutes ago   
3                                        Blood Hexes          30 minutes ago   
4  So..Vigilante. Can someone explain to me how i...   1 hour, 7 minutes ago   

       original_post_date  
0  Oct 23, 2009, 02:38 pm  
1  Jul 28, 2010, 12:42 pm  
2          12 minutes ago  
3        Sunday, 06:19 pm  
4      Saturday, 06:45 pm  
